In [1]:
from core.augment import create_augmented_dataset
from core.constants import AugmentationMethods, Datasets, TransformerModels
from core.loop import run_active_learning_loop
from datasets import load_dataset

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: You need to install sacremoses to use XLMTokenizer. See https://pypi.org/project/sacremoses/ for installation.

In [ ]:
def create_raw_set(
    dataset_name: str, augmentation_method: AugmentationMethods | None = None
):
    """If augmentation_method is not None, create an augmented dataset

    Args:
        dataset_name (str): Name of the dataset, string has to be a name, that is represented at huggingface.com/datasets
        augmentation_method (AugmentationMethods | None, optional): The augmentation method, available ones can be found at given enum. Defaults to None.

    Returns:
        tuple(dataset, dataset, dict): Raw sets and augmented indices, if augmentation_method is not None, else just the sets and an empty dict.
    """
    dataset = load_dataset(dataset_name)

    raw_test = dataset["test"]
    raw_train = dataset["train"]
    augmented_indices = {}
    if augmentation_method:
        raw_train, augmented_indices = create_augmented_dataset(
            raw_train, augmentation_method, n=num_augmentations
        )

    return raw_test, raw_train, augmented_indices
print("LOREM IPSUM")
raw_test, raw_train, augmented_indices = create_raw_set(Datasets.ROTTEN.value, AugmentationMethods.SYNONYM.value)

In [ ]:
query_strategies = [
    # Basic Strategies
    # "RandomSampling",
    # "BreakingTies",
    # Basic Augmented Strategies
    # "AugmentedSearchSpaceExtensionQueryStrategy",
    "AugmentedOutcomesQueryStrategy",
    # "AverageAcrossAugmentedQueryStrategy",
    # Combinations
    # "AugmentedSearchSpaceExtensionAndOutcomeQueryStrategy",
    # "AverageAcrossAugmentedExtendedOutcomesQueryStrategy",
]

datasets = [Datasets.ROTTEN.value]

num_queries = 50
num_samples = 20
num_augmentations = 5

run_active_learning_loop(
                raw_test,
                raw_train,
                augmented_indices,
                num_queries=num_queries,
                num_samples=num_samples,
                num_augmentations=num_augmentations
                if query_strategy != "BreakingTies"
                or query_strategy != "RandomSampling"
                else 0,
                query_strategy=query_strategy,
                model=TransformerModels.BERT.value,
                device="cuda",
            )